In [1]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
from matplotlib import pyplot as plt
import numpy as np
from utils import *
from NeuralNet import NN
from torch import nn



In [2]:
import torch, torchvision

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset_path = "data-science-summer-intern-2021/orders_autumn_2020.csv"
data = pd.read_csv(dataset_path).dropna()

add_hours_mins_cols(data)
add_distance_col(data)

print(data.columns)
X_names = ["ITEM_COUNT",
        "DISTANCES_KM",
        "ESTIMATED_DELIVERY_MINUTES",
        "WIND_SPEED", "TEMPERATURE"]

Y_names = ["ACTUAL_DELIVERY_MINUTES"]


X = data[X_names]
Y = data[Y_names]

(18429, 4)
[0.59968782 1.17992593 1.39110364 0.55631779 3.30509173]
Index(['DISTANCES_KM', 'TIMESTAMP_MINUTES', 'TIMESTAMP_HOURS', 'TIMESTAMP',
       'ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES', 'ITEM_COUNT',
       'USER_LAT', 'USER_LONG', 'VENUE_LAT', 'VENUE_LONG',
       'ESTIMATED_DELIVERY_MINUTES', 'ACTUAL_DELIVERY_MINUTES',
       'CLOUD_COVERAGE', 'TEMPERATURE', 'WIND_SPEED', 'PRECIPITATION'],
      dtype='object')


In [4]:

model = NN(5).to(device)

x_train, x_test = split_data(X.values, 0.7)
y_train, y_test = split_data(Y.values, 0.7)


y_test, y_val = split_data(y_test, 0.7)
x_test, x_val = split_data(x_test, 0.7)

print("y test len ", len(y_test))

def get_test_y_start_end(data_len, split_ratio=0.7):

    start = int(data_len * split_ratio)
    end = int(start + (data_len - start) * split_ratio)

    return start, end

y_estimate_start, y_estimate_end = get_test_y_start_end(len(Y.values), 0.7)

print("y estimate len ", y_estimate_end - y_estimate_start)
print(x_test.shape)
print(x_val.shape)
print(type(x_train))




y test len  3870
y estimate len  3870
(3870, 5)
(1659, 5)
<class 'numpy.ndarray'>


In [5]:
model.metric = nn.SmoothL1Loss()

In [6]:
model.train()
model.train_model(
    torch_np(x_train).to(device), 
    torch_np(y_train).to(device), 
    torch_np(x_val).to(device), 
    torch_np(y_val).to(device), 
    epochs=300, batch_size=128, validation_step=10)

e [  562 11327 11526 ... 12531  2080  1815]
 
 Validation loss:
tensor(6.8903, grad_fn=<SmoothL1LossBackward>) 

tensor(7.0161, grad_fn=<DivBackward0>)
shuffle [ 9021  8761  1882 ... 10956  4354  6360]
tensor(7.0452, grad_fn=<DivBackward0>)
shuffle [ 1625  3513  3405 ... 11910  5404  8336]
tensor(7.0301, grad_fn=<DivBackward0>)
shuffle [ 851  200 4529 ... 5880 9338 1295]
tensor(7.0352, grad_fn=<DivBackward0>)
shuffle [ 1222  7917 11018 ...  8826  7265    69]
tensor(7.0421, grad_fn=<DivBackward0>)
shuffle [12688  8632  8808 ...  5492  6622  8083]
tensor(7.0453, grad_fn=<DivBackward0>)
shuffle [ 1600   653  5955 ...  2997  2046 12182]
tensor(7.0179, grad_fn=<DivBackward0>)
shuffle [10392  2146  6256 ...  8923 11857  9182]
tensor(6.9612, grad_fn=<DivBackward0>)
shuffle [ 4046 10805    20 ...  9877  5155  3883]
tensor(7.0786, grad_fn=<DivBackward0>)
shuffle [11109  6724  8400 ...    99  7109  6659]
tensor(7.0300, grad_fn=<DivBackward0>)
shuffle [ 5993 11948 12109 ...  9097  5153  4694]
 
 

In [7]:
model.eval()
loss, pred = model.predict(torch.from_numpy(x_test.astype(np.float32)).to(device), torch.from_numpy(y_test.astype(np.float32)).to(device))
print(pred.cpu().detach().numpy() - y_test)
print(y_test)
print(loss)

[[ 0.05205917]
 [ 7.05205917]
 [ 2.05205917]
 ...
 [ 3.05205917]
 [ 9.11508179]
 [12.05205917]]
[[29]
 [22]
 [27]
 ...
 [26]
 [28]
 [17]]
tensor(6.7085, grad_fn=<SmoothL1LossBackward>)


In [8]:
# Is our predicition better than the provided estimate?
estimated_time = data["ESTIMATED_DELIVERY_MINUTES"].values[y_estimate_start: y_estimate_end]
provided_mse = mean_squared_error(y_test, estimated_time)
model_mse = mean_squared_error(y_test, pred.detach())

provided_mae = mean_absolute_error(y_test, estimated_time)
model_mae = mean_absolute_error(y_test, pred.detach())

print("provided mse ", provided_mse)
print("model mse", model_mse)

print("provided mae ", provided_mae)
print("model mae ", model_mae)

provided mse  85.88914728682171
model mse 76.87911194980073
provided mae  7.433333333333334
model mae  7.193802110046072


NN(
  (fc1): Linear(in_features=5, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop1): Dropout(p=0.6, inplace=False)
  (fc5): Linear(in_features=32, out_features=8, bias=True)
  (bn5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2): Dropout(p=0.6, inplace=False)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (metric): SmoothL1Loss()
)

In [ ]:
model = torchl.load("best_model")
loss, pred = model.predict(x_test, y_test)